<div style="text-align: center;"> 
  <h2><span style="color:#2C3E50;"><b>A/B TESTING OF ADS VS PSAS</b></span></h2> 
</div>

THIS NOTEBOOK CALCULATES THE **Z-STATISTIC FOR TWO PROPORTIONS**, **P-VALUE**, AND **CONFIDENCE INTERVALS** FOR COMPARING THE EFFECTIVENESS OF **ADVERTISEMENTS (ADS)** AND **PUBLIC SERVICE ANNOUNCEMENTS (PSAS)**.  

DETAILED ANALYSIS AND VISUALIZATIONS ARE AVAILABLE IN THE ACCOMPANYING DASHBOARD: [**DASHBOARD**](https://app.powerbi.com/reportEmbed?reportId=ae681465-0b11-445b-8fc9-b5bb2fcb9a77&autoAuth=true&ctid=d8d6e7dc-2b15-43c5-81fd-2efc145c9d2c)  

THE SCOPE OF THIS NOTEBOOK IS LIMITED TO STATISTICAL TESTING & COMPUTATIONS, INCLUDING:  

- **Z-TEST FOR TWO PROPORTIONS**  
- **P-VALUE**  
- **CONFIDENCE INTERVALS**  
- **CONVERSION RATES**  
- **ADDITIONAL STATISTICAL CALCULATIONS**  

---

### **<span style="color:#1ABC9C;">DATASET OVERVIEW</span>** 

- **USER ID**: UNIQUE IDENTIFIER FOR EACH USER  
- **TEST GROUP**: GROUP TO WHICH THE USER WAS EXPOSED (AD OR PSA)  
- **CONVERTED**: WHETHER THE USER CONVERTED OR NOT  
- **MOST ADS DAY**: THE DAY OF THE WEEK ON WHICH THE USER WAS SHOWN THE MOST ADS  
- **MOST ADS HOUR**: THE HOUR OF THE DAY DURING WHICH THE USER WAS SHOWN THE MOST ADS  
- **TOTAL ADS**: THE TOTAL NUMBER OF IMPRESSIONS TO WHICH THE USER WAS EXPOSED  

THE DATASET USED HERE IS PRECLEANED, SO WE WILL DIRECTLY START FROM STATISTICAL CALCULATIONS. THIS DATASET CAN BE DOWNLOADED FROM THE FOLLOWING LINK: [**DATASET**](https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing)  
___


### **<span style="color:#1ABC9C;">INITIALIZATION</span>** 

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import sqlalchemy
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
# Reading the table from MarketingCampaignDB from my sql server using sqlalchemy
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=KRISHNA\SQLEXPRESS;'
    r'DATABASE=MarketingCampaignDB;'
    r'Trusted_Connection=yes;'
)
conn = sqlalchemy.create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}').connect()
query = 'SELECT * FROM dbo.marketing_AB'

df = pd.read_sql(query, conn)

### **<span style="color:#1ABC9C;">RESAMPLING AND SPLITTING DATA INTO GROUPS</span>** 

In [2]:
# Splitting the df into ad group and psa group
ad_group = df[df['test_group'] == 'ad']
psa_group = df[df['test_group'] == 'psa']

print(len(ad_group))
print(len(psa_group))


564577
23524


In [3]:
# Since the ad group is 24x of psa group creatinf and huge imbalance we are
# Randomly sampling the ad group to match the size of the PSA group also reclaculating the n_ad 
n_psa = len(psa_group)
ad_group = ad_group.sample(n=n_psa, random_state=42)
n_ad = len(ad_group)
# Verifing the size of the sampled ad group
print(f"Sampled ad group size: {len(ad_group)}")


Sampled ad group size: 23524


In [4]:
# Joining both the groups into a single table to save it for future uses in power BI
sampled_df = pd.concat([ad_group, psa_group], ignore_index=True)
sampled_df.to_csv('sampled_data.csv', index=False)

### **<span style="color:#1ABC9C;">STATISTICAL ANALYSIS AND COMPUTATIONS</span>** 

In [5]:
# Counting the total conversion in each category since 'converted' column is a boolean using sum method 
converted_ad = ad_group['converted'].sum()
converted_psa = psa_group['converted'].sum()
print(f"Number of conversions in ad group: {converted_ad}")
print(f"Number of conversions in psa group: {converted_psa}")


Number of conversions in ad group: 618
Number of conversions in psa group: 420


In [6]:

# Calculating the conversion rate by computing the mean of the 'converted' column.  
# Since it contains boolean values, this provides the proportion of users who converted.
conv_rate_ad = converted_ad / n_ad
conv_rate_psa = converted_psa / n_psa
print(f"Conversion rate for ad group: {conv_rate_ad:.4f}")
print(f"Conversion rate for psa group: {conv_rate_psa:.4f}")

# Checking whether the difference is statistically significant.
# Using a two-proportion z-test since we’re comparing conversionS.
# For this, we need the counts of successes (conversions) and total observations for each group.

# If P-value is <0.05 we can easily conclude that the difference is not due to randomness or chance is statistically significant.
count = np.array([converted_ad, converted_psa])
nobs = np.array([n_ad, n_psa])
z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')
print(f"\nZ-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")


Conversion rate for ad group: 0.0263
Conversion rate for psa group: 0.0179

Z-statistic: 6.2146
P-value: 0.0000


In [7]:
# Getting 95% confidence intervals for each group’s conversion rate.
# This’ll show us the range where the true rates likely lie.
ci_ad = proportion_confint(converted_ad, n_ad, alpha=0.05, method='normal')
ci_psa = proportion_confint(converted_psa, n_psa, alpha=0.05, method='normal')
print(f"95% CI for ad group: {ci_ad}")
print(f"95% CI for psa group: {ci_psa}")


95% CI for ad group: (0.024227186780399392, 0.028314897899076884)
95% CI for psa group: (0.016161914715211324, 0.019546298173753137)


In [8]:
# Checking whether users in both groups were exposed to a similar number of Ads.  
# a large difference could introduce bias in the result.  
median_total_ads_ad = ad_group['total_ads'].median()
median_total_ads_psa = psa_group['total_ads'].median()
print(f"\nMedian total ads for ad group: {median_total_ads_ad:.2f}")
print(f"Median total ads for psa group: {median_total_ads_psa:.2f}")



Median total ads for ad group: 13.00
Median total ads for psa group: 12.00


### **<span style="color:#1ABC9C;">SAVING ALL THE ABOVE CALCULATIONS FOR FURTHER USE</span>** 

In [9]:
# Creating a DataFrame with the desired structure, including confidence intervals
powerbi_df = pd.DataFrame({
    'Group': ['ad', 'psa'],
    'Sample Size': [n_ad, n_psa],
    'Conversions': [converted_ad, converted_psa],
    'Conversion Rate': [conv_rate_ad, conv_rate_psa],
    '95% CI Lower': [ci_ad[0], ci_psa[0]],
    '95% CI Upper': [ci_ad[1], ci_psa[1]],
    'Z-Statistic': [z_stat, None], 
    'P-Value': [p_value, None]    
})

# Saving the DataFrame to a CSV file for PowerBI
powerbi_df.to_csv('campaign_statistical_metrics.csv', index=False)


___